In [1]:
def cmdir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def scaffold(gene):
    if gene != "None":
        try: return re.search("(.+?)_[0-9]+$", gene).group(1)
        except: print(gene)

def sbatch(name, cpus, cmd):
    return "sbatch -J %s -p serc -t 1- -c %d --mem %dG --wrap '%s'" %(name, cpus, cpus*8, cmd)

In [27]:
rootdir = "/scratch/users/ajaffe/deepeco/"
cmdir(rootdir)

## public genomes and metadata

In [8]:
cmdir(rootdir + "data")

### omd

wget https://sunagawalab.ethz.ch/share/microbiomics/ocean/suppl_data/genomes-fasta.tar.gz into data dir. download corresponding genome summary table from there or pub supp material.

In [4]:
omd = pd.read_csv(rootdir + "data/omd_summary.csv")
omd = omd[["Genome", "GTDB Taxonomy", "CheckM Completeness", "CheckM Contamination", "16S rRNA",
          "Genome size", "# scaffolds", "station", "depth", "depth layer",
          "latitude", "longitude", "detected in the water column"]]
omd.columns = ["genome_name", "classification", "checkm_completeness", "checkm_redundancy",
              "16S_rRNA", "genome_length", "num_scaffolds", "station",
              "depth", "depth_layer", "lat", "long", "detected_water_column"]
omd.head(2)

,genome_name,classification,checkm_completeness,checkm_redundancy,16S_rRNA,genome_length,num_scaffolds,station,depth,depth_layer,lat,long,detected_water_column
0,MARD_SAMEA4707921_REFG_MMP4707921,d__Bacteria;p__Cyanobacteria;c__Cyanobacteriia...,100.00,0.41,1.0,3057835,35,NaN,NaN,NaN,NaN,NaN,True
1,TARA_SAMEA2623741_METAG_DLKGCKBM,d__Bacteria;p__Eremiobacterota;c__UBP9;o__UBA4...,97.22,11.88,0.0,9451077,96,NaN,NaN,NaN,NaN,NaN,True


### odna

wget representative and non representative genomes from here, as well as supp table: https://springernature.figshare.com/collections/The_OceanDNA_MAG_catalog_contains_over_50_000_prokaryotic_genomes_originated_from_various_marine_environments/5564844

In [5]:
odna = pd.read_csv(rootdir + "data/odna.tsv", sep="\t")
odna = odna[["genome", "division", "biosample", "num_contig", "is_16S_rRNA_detected",
            "checkm_compl", "checkm_contam", "gtdb_classification",
            "division", "subdivision", "sample_name", "sra_sample",
            "original_bioproject", "original_biosample", "sra_run",
            "depth", "latitude", "longigute"]].rename(columns={"longigute":"longitude"})
odna.head(2)

,genome,division,biosample,num_contig,is_16S_rRNA_detected,checkm_compl,checkm_contam,gtdb_classification,division,subdivision,sample_name,sra_sample,original_bioproject,original_biosample,sra_run,depth,latitude,longitude
0,OceanDNA-a1,Tara girus,SAMD00334869,83,no,69.78,0.00,d__Archaea;p__Aenigmatarchaeota;c__Aenigmatarc...,Tara girus,TARA_Girus,ERS488773,ERS488773,PRJEB1788,SAMEA2619974,"ERR594290,ERR594345",600.0,20.83220,63.6004
1,OceanDNA-a2,Tara prok,SAMD00334870,374,yes,62.72,0.07,d__Archaea;p__Asgardarchaeota;c__Heimdallarcha...,Tara prok,Sunagawa_et_al_2015,ERS493914,ERS493914,PRJEB1787,SAMEA2623446,ERR599029,5.0,9.84705,-80.0530


### other pubs

In [6]:
# omz catalog - https://www.nature.com/articles/s41597-023-02222-y
omz = pd.read_csv(rootdir + "data/omz_catalog.csv")
omz.head(2)

,Bin Id,Method,Marker lineage,# genomes,# markers,# marker sets,Completeness,Contamination,Strain heterogeneity,Genome size (bp),...,GC std (scaffolds > 1kbp),Coding density,Translation table,# predicted genes,0,1,2,3,4,5+
0,AAA001_A19,CheckM v1.2.1,k__Archaea (UID2),207.0,145.0,103.0,27.18,0.0,0.0,446988.0,...,1.56,92.82,11.0,570.0,103.0,42.0,0.0,0.0,0.0,0.0
1,AAA001_B15,CheckM v1.2.1,p__Proteobacteria (UID3880),1495.0,242.0,151.0,25.57,0.0,0.0,488860.0,...,2.04,93.73,11.0,493.0,166.0,76.0,0.0,0.0,0.0,0.0


Plus two others (Zhang, Sun et al. + Sun and Ward).

## identify genomes of interest

In [21]:
cmdir(rootdir + "scripts")
cmdir(rootdir + "graft")

### run graft per genome

In [13]:
# set up
genome_dirs = glob.glob(rootdir + "data/*/")
genome_dirs.append("/scratch/groups/dekas/OC17_metagenomics_2022/genomes/refined_genomes/")
gpkg = "/home/users/ajaffe/misc/graftm_gpkg/rubisco_superfamily.gpkg/"

In [33]:
# which are unrun?

genomes = []
completed = [item.split("/")[-2] for item \
             in glob.glob(rootdir + "graft/chunk*/*/")]
    
for genome_dir in genome_dirs:
    for gpath in glob.glob(genome_dir + "/*"):
        genome = re.search("(.+?)\.f[na\.gz]", os.path.basename(gpath)).group(1)
        if genome not in completed:
            genomes.append(gpath)

In [26]:
count, chunk = 0, 0
current=[]
out = open(rootdir + "/scripts/graft.sh", "w")

for genome in genomes:
    
    if count < round(len(genomes),0)/50:
        current.append(genome)
        count+=1
    
    else:
        
        outdir = rootdir + "graft/chunk" + str(chunk)
        base = "graftM graft --threads 20 --evalue 1e-5 --forward %s \
                --input_sequence_type nucleotide --search_only --graftm_package %s \
                --output_directory %s" %(" ".join(current), gpkg, outdir)
        out.write(sbatch("graft", 20, base) + "\n")
        print(len(current))
        count=0
        chunk+=1
        current=[]

out.close()

179


### parse grafts

In [6]:
rub_info = defaultdict(list)

for graftdir in glob.glob(rootdir + "graft/chunk*/*/"):

    name = graftdir.split("/")[-2]
    aln = glob.glob(graftdir + "*orf.fa")
    
    if aln != []:
        
        for record in sfp(open(aln[0])):
            rub_info["genome"].append(name)
            rub_info["protein"].append(record[0].split(" ")[0])
            rub_info["protein_len"].append(len(record[1]))
            rub_info["protein_seq"].append(str(record[1]))

rub_df = pd.DataFrame(rub_info)
rub_df.head()

,genome,protein,protein_len,protein_seq
0,OceanDNA-b43099,OceanDNA-b43099_00060_1753_4_38,478,AGRAIMAKKGYDAGVKEYRDTYWMPEYEPKDTDILACFKITPQPGV...
1,OceanDNA-b41123,OceanDNA-b41123_00033_5629_1_112,285,CPRQSLPRAACAPGPGLQLLRSGGMSAPIGVFDSGVGGLSVLREIR...
2,OceanDNA-b17538,OceanDNA-b17538_00047_3787_1_75,221,SKVMKIAPSILSCDFSRLGEEIVAVKEGGADWIHVDVMDGHFVPNI...
3,OceanDNA-b38333,OceanDNA-b38333_00008_26579_2_507,423,LMSTRIHARYLIETPYPVEHAAQVMAGEQSSGTFTRLANETDALRA...
4,OceanDNA-b13241,OceanDNA-b13241_00184_4013_5_57,36,CLAVKQCLFLHVKKATREPWQVLRKCLPIPSSGSQL


### clean up proteins

In [36]:
cmdir(rootdir + "proteins")

In [9]:
def parse_kofamscan(path):
   
    buffer = []
    for line in open(path).readlines():
        if "#" not in line:
            # hilariously long regex
            m = re.search("[* ]*([\S]+)\s+([\S]+)\s+([0-9.-]+)\s+" + \
                "([0-9.-]+)\s+([0-9.+-e]+)\s(.+?$)", line.strip())
            try:
                buffer.append(m.groups())
            except:
                print(line)

    kodf = pd.DataFrame.from_records(buffer, columns =["gene", "ko", "threshold", "score", "eval", "def"]) 
    buffer=[]
    return kodf

In [38]:
with open(rootdir + "proteins/first_pass.faa", "w") as outfile:
    for key, row in rub_df.iterrows():
        outfile.write(">%s\n%s\n" %(row["protein"], row["protein_seq"]))

In [39]:
# launch kofamscan
kocall = "exec_annotation -o %s %s -p %s -k %s --cpu 20 -f detail" \
    %(rootdir + "proteins/first_pass.kfscan.txt", rootdir + "proteins/first_pass.faa",
     "/home/groups/dekas/software/kofamscan/profiles/prokaryote.hal",
      "/home/groups/dekas/software/kofamscan/ko_list")
print(sbatch("kofamscan", 20, kocall))

sbatch -J kofamscan -p serc -t 1- -c 20 --mem 160G --wrap 'exec_annotation -o /scratch/users/ajaffe/deepeco/proteins/first_pass.kfscan.txt /scratch/users/ajaffe/deepeco/proteins/first_pass.faa -p /home/groups/dekas/software/kofamscan/profiles/prokaryote.hal -k /home/groups/dekas/software/kofamscan/ko_list --cpu 20 -f detail'


In [10]:
# filter for significance
kodf = parse_kofamscan(rootdir + "proteins/first_pass.kfscan.txt")
kodf["eval"] = kodf["eval"].apply(lambda x: float(x))
kodf["score"] = kodf["score"].apply(lambda x: float(x))
kodf = kodf[kodf["eval"] < 1e-5]
# get best hit per gene based on score
kfilt = kodf.sort_values('score', ascending=False).drop_duplicates("gene")
kfilt["ko"].value_counts()[0:10]

K01601    4868
K01698    4740
K00788    1632
K01559    1003
K25035     896
K01783     511
K13038     249
K01714     245
K01776     209
K03972     149
Name: ko, dtype: int64

In [11]:
# main rubisco ko + 2 RLP ones
genes_to_retain = kfilt[kfilt["ko"].isin(["K01601", "K08965", "K25035"])]["gene"].to_list()
rub_df_filtered = rub_df[rub_df["protein"].isin(genes_to_retain)]
rub_df_filtered.head()

,genome,protein,protein_len,protein_seq
0,OceanDNA-b43099,OceanDNA-b43099_00060_1753_4_38,478,AGRAIMAKKGYDAGVKEYRDTYWMPEYEPKDTDILACFKITPQPGV...
13,OceanDNA-b43459,OceanDNA-b43459_00039_2973_3_74,464,KAVSLNQSSRYADLNLDEQTLMDEGRHILCAYRMKPKPGAGYLEGA...
15,OceanDNA-b35742,OceanDNA-b35742_00039_3392_2_73,500,TSKRFTVNYTHPPLQFNALDERTHLEQIFFSPYRLNFQENGMDQSA...
16,OceanDNA-b35742,OceanDNA-b35742_00388_705_3_27,403,SSPTKVQRTGWGYPTGKIFFILTDLNFLENGMDQSARYADLSLDED...
20,OceanDNA-b16381,OceanDNA-b16381_00013_5941_1_86,305,QELSMAKKYDAGVKEYRDTYFTPDYVPLDTDLLACFKCTGQEGVPK...


### write out working set

In [42]:
cmdir(rootdir + "genomes")
cmdir(rootdir + "genomes/first_pass")

In [14]:
paths = {}

for genome_dir in genome_dirs: 
    for genome in glob.glob(genome_dir + "/*"):
        name = re.search("(.+?)\.f[na\.gz]", os.path.basename(genome)).group(1)
        paths[name] = genome
        
rub_df_filtered["gpath"] = rub_df_filtered["genome"].map(paths).fillna("None")

/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
for key, row in rub_df_filtered.iterrows():
    
    with open(rootdir + "genomes/first_pass/" + row["genome"] + ".fa", "w") as out:
        
        if ".gz" in row["gpath"]:
            with gzip.open(row["gpath"], "rt") as handle:
                for record in sfp(handle):
                    out.write(">%s\n%s\n" %(record[0], str(record[1])))
        else:
            for record in sfp(open(row["gpath"])):
                out.write(">%s\n%s\n" %(record[0], str(record[1])))  

In [45]:
cmd = "rclone copy /scratch/users/ajaffe/deepeco/genomes/first_pass drive:deepeco/first_pass_genomes --progress"
print(cmd)

rclone copy /scratch/users/ajaffe/deepeco/genomes/first_pass drive:deepeco/first_pass_genomes --progress


## generate genome metadata

In [46]:
cmdir(rootdir + "genomes/tk/")
cmdir(rootdir + "genomes/quality/")

### identity

In [47]:
genomes = glob.glob(rootdir + "genomes/first_pass/*")
n = math.ceil(len(genomes)/20)

for a, i in enumerate(range(0, len(genomes), n)):
    with open(rootdir + "genomes/tk/batch%d.txt" %(a), "w") as out:
        for genome in genomes[i:i + n]:
            out.write(genome + "\t" + os.path.basename(genome).split(".fa")[0] + "\n")

In [48]:
with open(rootdir + "genomes/tk/classify.sh", "w+") as out:
    for batchfile in glob.glob(rootdir + "genomes/tk/batch*.txt"):
        dirname = rootdir + "genomes/tk/%s" %(os.path.basename(batchfile).replace(".txt", ""))
        base = "gtdbtk classify_wf --cpus 20 -x .fa --batchfile %s --out_dir %s" %(batchfile, dirname)
        cmd = sbatch(os.path.basename(batchfile).split(".")[0], 20, base)
        out.write(cmd + "\n")

In [13]:
# parse
gtresults = pd.concat([pd.read_csv(item, sep="\t") for item in glob.glob(rootdir + "genomes/tk/batch*/gtdbtk.*.summary.tsv")])
gtresults.head(2)

,user_genome,classification,fastani_reference,fastani_reference_radius,fastani_taxonomy,fastani_ani,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,pplacer_taxonomy,classification_method,note,"other_related_references(genome_id,species_name,radius,ANI,AF)",msa_percent,translation_table,red_value,warnings
0,GORG_SAMEA6075888_SAGS_AG414B09,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,GCF_000812185.1,95.0,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,96.14,0.87,GCF_000812185.1,95.0,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,96.14,0.87,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,taxonomic classification defined by topology a...,topological placement and ANI have congruent s...,"GCA_902552015.1, s__Nitrosopelagicus sp9025520...",68.95,11.0,NaN,NaN
1,HOTS_SAMN07137057_METAG_MGKKCAMH,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,NaN,NaN,NaN,NaN,NaN,GCA_002495905.1,95.0,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,88.30,0.80,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...,taxonomic classification defined by topology a...,NaN,"GCA_013378385.1, s__UBA57 sp013378385, 95.0, 8...",89.21,11.0,0.983248,Genome not assigned to closest species as it f...


In [14]:
# how many are CPR-DPANN?
dpann_phyla = ["p__Nanoarchaeota", "p__Huberarchaeota", "p__UAP2", "p__EX4484-52", 
               "p__Nanohaloarchaeota", "p__Iainarchaeota", "p__Aenigmatarchaeota", "p__Micrarchaeota"]
gtresults["phylum"] = gtresults["classification"].apply(lambda x: x.split(";")[1] if len(x.split(";"))>1 else "None")
gtresults[(gtresults["phylum"].str.contains("Patesci") | gtresults["phylum"].isin(dpann_phyla))]["phylum"].value_counts()

p__Nanoarchaeota        216
p__Patescibacteria       19
p__Iainarchaeota          5
p__Aenigmatarchaeota      3
Name: phylum, dtype: int64

### quality

In [49]:
with open(rootdir + "genomes/quality/run_checkm.sh", "w") as out:
    
    for batchfile in glob.glob(rootdir + "genomes/tk/batch*.txt"):
        
        # create subdir for each batch
        basename = os.path.basename(batchfile).split(".")[0]
        batchdir = rootdir + "genomes/quality/" + basename + "/"
        cmdir(batchdir)
        
        # rewrite batch file to work with checkm
        with open(batchdir + "batchfile.txt", "w") as bf:
            for line in open(batchfile).readlines():
                bf.write("%s\t%s\n" %(line.strip().split("\t")[1],
                    line.strip().split("\t")[0]))
        
        # construct checkm calls
        call = "checkm lineage_wf -t 20 -x .fa --pplacer_threads 20 %s %s" %(batchdir + "batchfile.txt", batchdir)
        call2 = "checkm qa -t 20 -o 1 -f %s --tab_table %s %s" %(batchdir + "output_table.txt",
            batchdir + "lineage.ms", batchdir)
        cmd = sbatch(basename, 20, "%s && %s" %(call, call2))
        out.write(cmd + "\n")

In [16]:
# read in all results
checkm_df = pd.concat([pd.read_csv(item, sep="\t") for item in 
    glob.glob(rootdir + "genomes/quality/batch*/output_table.txt")]).reset_index()
checkm_df = checkm_df[["Bin Id", "Completeness", "Contamination"]]
checkm_df.columns = ["user_genome", "checkm_completeness", "checkm_redundancy"]
checkm_df.head()

,user_genome,checkm_completeness,checkm_redundancy
0,AB-750_A23_AB-903_clean,40.15,0.86
1,AB-750_C07_AB-903_clean,32.60,1.87
2,AB-750_I16_AB-904_clean,22.75,0.00
3,AB-750_K03_AB-904_clean,51.73,1.38
4,AB-754_I23_AB-906_clean,40.87,1.49


### redundancy - first pass

In [56]:
cmdir(rootdir + "genomes/first_pass_drep/")

In [57]:
# reformat quality information
drep_df = checkm_df.rename(columns={"checkm_completeness":"completeness","checkm_redundancy":"contamination"})
drep_df["genome"] = drep_df["user_genome"].apply(lambda x: x + ".fa")
drep_df[["genome", "completeness", "contamination"]].to_csv(rootdir + "genomes/first_pass_drep/genomeInformation.csv", sep=",", index=False)

In [58]:
call = "dRep dereplicate %s -sa 0.95 -p 20 -comp 50 -con 25 -d -g %s --genomeInfo %s" %(rootdir + \
    "/genomes/first_pass_drep/", rootdir + "/genomes/first_pass/*fa",rootdir + "genomes/first_pass_drep/genomeInformation.csv")
cmd = sbatch("drep", 20, call)
print(cmd)

sbatch -J drep -p serc -t 1- -c 20 --mem 160G --wrap 'dRep dereplicate /scratch/users/ajaffe/deepeco//genomes/first_pass_drep/ -sa 0.95 -p 20 -comp 50 -con 25 -d -g /scratch/users/ajaffe/deepeco//genomes/first_pass/*fa --genomeInfo /scratch/users/ajaffe/deepeco/genomes/first_pass_drep/genomeInformation.csv'


In [18]:
print("prelim dRep and quality filtering cut down %d bins to %d." %(len(glob.glob(rootdir+"genomes/first_pass/*")), len(glob.glob(rootdir+"genomes/first_pass_drep/dereplicated_genomes/*"))))

prelim dRep and quality filtering cut down 5113 bins to 1192.


## genome refinement

In [60]:
cmdir(rootdir + "genomes/contigs")
cmdir(rootdir + "genomes/contigs/fasta")

### get contigs across genomes

In [61]:
count = 0

for genome in rub_df_filtered["genome"].unique():
    
    subtable = rub_df_filtered[rub_df_filtered["genome"]==genome]
    gpath = subtable["gpath"].iloc[0]
    proteins = subtable["protein"].to_list()
    contigs = ["_".join(protein.split("_")[:-3]) for protein in proteins]
    count += len(set(contigs))
        
    if ".gz" in gpath:
        handle = gzip.open(gpath, "rt")
    else: handle = open(gpath)
    
    for record in sfp(handle):
        if record[0].split(" ")[0] in contigs:
            clean = record[0].split(" ")[0].replace("(", "_").replace(")", "_")
            with open(rootdir + "genomes/contigs/fasta/" + clean + ".fna", "w") as out:
                out.write(">%s\n%s\n" %(record[0].split(" ")[0], str(record[1])))

print(count)

5747


### annotate proteins

In [29]:
cmdir(rootdir + "genomes/contigs/diamond")
cmdir(rootdir + "genomes/contigs/diamond/parts")
adir = rootdir + "genomes/contigs/"
import skbio

In [62]:
with open(rootdir + "genomes/contigs/predict.sh", "w") as out:
    for contig in glob.glob(rootdir + "genomes/contigs/fasta/*"):
        call = "prodigal -i %s -a %s -p meta" %(contig, contig.replace(".fna", ".faa"))
        out.write(call + "\n")

cat all prodigal into all_scaffold_proteins.faa.

In [66]:
# split and launch diamond
records = [r for r in sfp(open(adir + "diamond/all_scaffold_proteins.faa"))]
wrapper = open(adir + "diamond/diamond.sh", "w")

n = math.ceil(len(records)/30)
for i in range(0, len(records),n):
    
    # write partial faa
    with open(adir + "diamond/parts/part" + str(int(i/n)+1) + ".faa", "w") as chunk:
        for record in records[i:i + n]:
            chunk.write(">%s\n%s\n" %(record[0], str(record[1])))
    
    # generate diamond call
    call = "diamond blastp -d /$OAK/db/uniref100.dmnd -q %s -o %s --threads 20 -b8 -c1" \
            %(adir + "diamond/parts/part" + str(int(i/n)+1) + ".faa",
              adir + "diamond/parts/part" + str(int(i/n)+1) + ".txt")
    cmd = sbatch("dmnd", 20, call)
    wrapper.write(cmd + "\n")

wrapper.close()

In [37]:
# concatenate + collect taxonomy
dmnd = pd.concat([skbio.io.read(item, format="blast+6", into=pd.DataFrame, default_columns=True) \
                      for item in glob.glob(adir + "diamond/parts/part*.txt")])
# compute coverage
faalens = {record[0].split(" ")[0]: len(record[1]) for record \
           in sfp(open(adir + "diamond/all_scaffold_proteins.faa"))}
dmnd["qlen"] = dmnd["qseqid"].map(faalens)
dmnd["qcov"] = dmnd.apply(lambda x: (x["qend"]-x["qstart"])/x["qlen"], axis=1)
# choose best hits for each
dmnd = dmnd.sort_values(["bitscore", "qcov"], ascending=[False,False]).drop_duplicates("qseqid")
# filter for min cov /eval
dmnd = dmnd[(dmnd["evalue"]<1e-20) & (dmnd["qcov"]>0.70)]
dmnd.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,qcov
254960,GORG_AG893I20_SAGS-scaffold_1_86,UniRef100_A0A3D8UNY3,98.3,1501.0,25.0,0.0,1.0,1501.0,1.0,1501.0,0.0,2922.0,1502,0.998668
202970,GORG_AG891C19_SAGS-scaffold_1_60,UniRef100_A0A3D8UNY3,98.3,1501.0,26.0,0.0,1.0,1501.0,1.0,1501.0,0.0,2920.0,1502,0.998668
184217,GORG_AG447F10_SAGS-scaffold_1_110,UniRef100_A0A3D8UNY3,98.3,1501.0,25.0,0.0,1.0,1501.0,1.0,1501.0,0.0,2919.0,1502,0.998668
189199,GORG_AG453A13_SAGS-scaffold_1_48,UniRef100_A0A0A2B5Q5,93.6,1298.0,83.0,0.0,1.0,1298.0,1.0,1298.0,0.0,2365.0,1299,0.998460
225892,GORG_AG892J14_SAGS-scaffold_1_59,UniRef100_A0A8I1WZA4,91.2,1298.0,112.0,2.0,1.0,1296.0,1.0,1298.0,0.0,2303.0,1297,0.998458


### get taxonomy

In [32]:
from scipy import stats
from bs4 import BeautifulSoup

In [19]:
# map to uniref ids
with open(adir + "diamond/uniref_ids.txt", "w") as out:
    for uid in dmnd["sseqid"].unique():
        out.write(uid.split("_")[1] + "\n")

N.B. need to do two searches, one for uniref and one for uniparc. In this case also need to split input.

In [38]:
uni = pd.concat([pd.read_csv(item,sep="\t", dtype=str) for \
                 item in glob.glob(adir + "diamond/uniref/*tsv")]).fillna("None")
uni.columns = ["from", "entry", "organism", "organism_id"]
uni["taxid"] = uni["organism_id"].apply(lambda x: stats.mode(x.split("; "))[0][0])
uni.head()

,from,entry,organism,organism_id,taxid
0,UPI0008E7A175,A0A9W5XJD4,Micromonospora sediminimaris,547162,547162
1,UPI0008DF770B,A0A9W5XHW8,Micromonospora sediminimaris,547162,547162
2,UPI0008EAC30F,A0A9W5XHX2,Micromonospora sediminimaris,547162,547162
3,UPI0008F0A88F,A0A9W5XI89,Micromonospora sediminimaris,547162,547162
4,UPI0008EDCAD6,A0A9W5ULQ2,Micromonospora sediminimaris,547162,547162


In [23]:
with open(adir + "diamond/ncbi_tax.txt", "w") as out:
    for tax in uni["taxid"].unique():
        if tax != "None":
            out.write(tax + "\n")

In [39]:
lineage_info = defaultdict(list)

for xml in glob.glob(adir + "/diamond/*xml"):
    
    for block in BeautifulSoup(open(xml), "xml").findAll("Taxon"):
        
        lineage, phylum, classe = "None", "None", "None"

        if block.find("Lineage"):
            lineage = block.find("Lineage").string

        for level in block.findAll("Taxon"):
            if level.find("Rank").string=="phylum":
                phylum = level.find("ScientificName").string
            elif level.find("Rank").string=="class":
                classe = level.find("ScientificName").string
                
        if (phylum == classe == 'None') and (lineage!='None'):
            phylum = lineage.split("; ")[-1]

        lineage_info["taxid"].append(block.find("TaxId").string)
        lineage_info["lineage"].append(lineage)
        lineage_info["phylum"].append(";".join([phylum, classe]))

lineage_df = pd.DataFrame(lineage_info).query("lineage!='None'")
uni = uni.merge(lineage_df, how="left", on="taxid")
uni.head()

,from,entry,organism,organism_id,taxid,lineage,phylum
0,UPI0008E7A175,A0A9W5XJD4,Micromonospora sediminimaris,547162,547162,cellular organisms; Bacteria; Terrabacteria gr...,Actinomycetota;Actinomycetes
1,UPI0008DF770B,A0A9W5XHW8,Micromonospora sediminimaris,547162,547162,cellular organisms; Bacteria; Terrabacteria gr...,Actinomycetota;Actinomycetes
2,UPI0008EAC30F,A0A9W5XHX2,Micromonospora sediminimaris,547162,547162,cellular organisms; Bacteria; Terrabacteria gr...,Actinomycetota;Actinomycetes
3,UPI0008F0A88F,A0A9W5XI89,Micromonospora sediminimaris,547162,547162,cellular organisms; Bacteria; Terrabacteria gr...,Actinomycetota;Actinomycetes
4,UPI0008EDCAD6,A0A9W5ULQ2,Micromonospora sediminimaris,547162,547162,cellular organisms; Bacteria; Terrabacteria gr...,Actinomycetota;Actinomycetes


In [40]:
# add back in
dmnd["scaffold"] = dmnd["qseqid"].apply(lambda x: "_".join(x.split("_")[:-1]))
dmnd["entry"] = dmnd["sseqid"].apply(lambda x: x.split("_")[1])
dmnd = dmnd.merge(uni[["entry", "phylum"]], how="left").fillna("None")
dmnd.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,qcov,scaffold,entry,phylum
0,GORG_AG893I20_SAGS-scaffold_1_86,UniRef100_A0A3D8UNY3,98.3,1501.0,25.0,0.0,1.0,1501.0,1.0,1501.0,0.0,2922.0,1502,0.998668,GORG_AG893I20_SAGS-scaffold_1,A0A3D8UNY3,unclassified Bacteria;None
1,GORG_AG891C19_SAGS-scaffold_1_60,UniRef100_A0A3D8UNY3,98.3,1501.0,26.0,0.0,1.0,1501.0,1.0,1501.0,0.0,2920.0,1502,0.998668,GORG_AG891C19_SAGS-scaffold_1,A0A3D8UNY3,unclassified Bacteria;None
2,GORG_AG447F10_SAGS-scaffold_1_110,UniRef100_A0A3D8UNY3,98.3,1501.0,25.0,0.0,1.0,1501.0,1.0,1501.0,0.0,2919.0,1502,0.998668,GORG_AG447F10_SAGS-scaffold_1,A0A3D8UNY3,unclassified Bacteria;None
3,GORG_AG453A13_SAGS-scaffold_1_48,UniRef100_A0A0A2B5Q5,93.6,1298.0,83.0,0.0,1.0,1298.0,1.0,1298.0,0.0,2365.0,1299,0.998460,GORG_AG453A13_SAGS-scaffold_1,A0A0A2B5Q5,Cyanobacteriota;Cyanophyceae
4,GORG_AG892J14_SAGS-scaffold_1_59,UniRef100_A0A8I1WZA4,91.2,1298.0,112.0,2.0,1.0,1296.0,1.0,1298.0,0.0,2303.0,1297,0.998458,GORG_AG892J14_SAGS-scaffold_1,A0A8I1WZA4,Cyanobacteriota;Cyanophyceae


### reconcile

In [26]:
orf_counts = {}

for record in sfp(open(adir + "diamond/all_scaffold_proteins.faa")):
    scaf = "_".join(record[0].split(" ")[0].split("_")[:-1])
    if scaf not in orf_counts:
        orf_counts[scaf] = 1
    else: orf_counts[scaf] +=1
    
len(orf_counts.keys())

5747

In [27]:
tax_info = defaultdict(list)

for scaf in dmnd["scaffold"].unique():
    
    subtable = dmnd[dmnd["scaffold"]==scaf].groupby(["scaffold", "phylum",], \
        as_index=False).aggregate({"qseqid":"count"}).sort_values("qseqid", ascending=False)
    subtable["total_orfs"] = orf_counts[subtable["scaffold"].iloc[0]]
    subtable["perc_orfs"] = subtable.apply(lambda x: x["qseqid"]/x["total_orfs"], axis=1)
    sorted_table = subtable.sort_values("perc_orfs", ascending=False)
    tax_info["scaffold"].append(sorted_table["scaffold"].iloc[0])
    tax_info["prot_phylum_winner"].append(sorted_table["phylum"].iloc[0])
    tax_info["prot_phylum_winner_perc"].append(sorted_table["perc_orfs"].iloc[0])

tax_info_df = pd.DataFrame(tax_info)
tax_info_df.tail()

,scaffold,prot_phylum_winner,prot_phylum_winner_perc
5741,AB-746_K17_AB-902_NODE_385_length_285_cov_0.91...,Pseudomonadota;Gammaproteobacteria,1.0
5742,AB-747_D03_AB-902_NODE_602_length_208_cov_1.11...,Pseudomonadota;Gammaproteobacteria,1.0
5743,TARA_SAMEA2623295_METAG-scaffold_168437,Planctomycetota;Candidatus Brocadiia,1.0
5744,MARD_MMP02671383_SAGS-scaffold_187,Cyanobacteriota;Cyanophyceae,1.0
5745,AB-750_C07_AB-903_NODE_1237_length_210_cov_2.0...,None,1.0


### curate

In [96]:
tmerge = tax_info_df

tmp = {}
for contig in glob.glob(adir + "fasta/*"):
    for record in sfp(open(contig)):
        tmp[record[0]]=len(record[1])
tmerge["scaffold_len"] = tmerge["scaffold"].map(tmp)
tmp = {}

In [97]:
scaf2bin = {}

for genome in glob.glob(rootdir + "genomes/first_pass/*"):
    for record in sfp(open(genome)):
        scaf2bin[record[0].split(" ")[0]] = os.path.basename(genome).replace(".fa", "")

tmerge["bin"] = tmerge["scaffold"].map(scaf2bin)
tmerge = tmerge.merge(gtresults[["user_genome", "classification"]], how="left", left_on="bin", right_on="user_genome")
tmerge.head()

,scaffold,prot_phylum_winner,prot_phylum_winner_perc,scaffold_len,bin,user_genome,classification
0,MARD_MMP02436139_REFG-scaffold_1,Cyanobacteriota;Cyanophyceae,0.886944,2584918,MARD_SAMN02436139_REFG_MMP02436139,MARD_SAMN02436139_REFG_MMP02436139,d__Bacteria;p__Cyanobacteria;c__Cyanobacteriia...
1,MARD_MMP02261338_REFG-scaffold_1,None,0.916648,5066637,MARD_SAMN02261338_REFG_MMP02261338,MARD_SAMN02261338_REFG_MMP02261338,d__Bacteria;p__Cyanobacteria;c__Cyanobacteriia...
2,MARD_MMP02436142_REFG-scaffold_6,Pseudomonadota;Alphaproteobacteria,0.936694,2598952,MARD_SAMN02436142_REFG_MMP02436142,MARD_SAMN02436142_REFG_MMP02436142,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...
3,OceanDNA-b22055_00001,Planctomycetota;Candidatus Brocadiia,0.960000,148688,OceanDNA-b22055,OceanDNA-b22055,d__Bacteria;p__Planctomycetota;c__Brocadiae;o_...
4,OceanDNA-b22057_00001,Planctomycetota;Candidatus Brocadiia,0.960000,150137,OceanDNA-b22057,OceanDNA-b22057,d__Bacteria;p__Planctomycetota;c__Brocadiae;o_...


In [98]:
# bring in genome clustering info
cclust = pd.read_csv(rootdir + "genomes/first_pass_drep/data_tables/Cdb.csv")
cclust["bin"] = cclust["genome"].apply(lambda x: x.replace(".fa", ""))
# right merge to include only clustered/quality genomes from above - dont freak out!
tmerge = tmerge.merge(cclust[["bin", "primary_cluster","secondary_cluster"]], how="right", on="bin")
tmerge = tmerge[["scaffold", "scaffold_len", "primary_cluster", "secondary_cluster", \
        "prot_phylum_winner", "prot_phylum_winner_perc", "classification"]].sort_values(["primary_cluster", \
        "secondary_cluster", "scaffold_len"], ascending=[True,True,False]).to_csv(adir + "diamond/to_curate.csv", index=False)

In [99]:
# read in curated version
curated = pd.read_csv(adir + "diamond/curated.tsv", sep="\t").fillna(False).replace("x", True)
curated.head(2)

,scaffold,scaffold_len,primary_cluster,secondary_cluster,prot_phylum_winner,prot_phylum_winner_perc,classification,misbinned
0,JASMDC010000008.1,111644,1,1_1,Candidatus Bathyarchaeota;None,0.044248,d__Archaea;p__SpSt-1190;c__SpSt-1190;o__;f__;g...,False
1,JASMIZ010000144.1,6555,1,1_1,Acidobacteriota;None,0.250000,d__Archaea;p__SpSt-1190;c__SpSt-1190;o__;f__;g...,True


## make second pass set

In [122]:
cmdir(rootdir + "genomes/second_pass")
cmdir(rootdir + "genomes/second_pass_drep")

In [123]:
# filter based on genome quality, misbinning status, and scaffold length
scaffolds_to_retain = curated.query("misbinned==False").query("scaffold_len>=1500")["scaffold"].to_list()
rub_df_filtered["scaffold"] = rub_df_filtered["protein"].apply(lambda x: "_".join(x.split("_")[:-3]))
rub_df_refiltered = rub_df_filtered[rub_df_filtered["scaffold"].isin(scaffolds_to_retain)]
rub_df_refiltered.head()

/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,genome,protein,protein_len,protein_seq,gpath,scaffold
0,OceanDNA-b43099,OceanDNA-b43099_00060_1753_4_38,478,AGRAIMAKKGYDAGVKEYRDTYWMPEYEPKDTDILACFKITPQPGV...,/scratch/users/ajaffe/deepeco/data/odna_redund...,OceanDNA-b43099_00060
13,OceanDNA-b43459,OceanDNA-b43459_00039_2973_3_74,464,KAVSLNQSSRYADLNLDEQTLMDEGRHILCAYRMKPKPGAGYLEGA...,/scratch/users/ajaffe/deepeco/data/odna_redund...,OceanDNA-b43459_00039
15,OceanDNA-b35742,OceanDNA-b35742_00039_3392_2_73,500,TSKRFTVNYTHPPLQFNALDERTHLEQIFFSPYRLNFQENGMDQSA...,/scratch/users/ajaffe/deepeco/data/odna_redund...,OceanDNA-b35742_00039
16,OceanDNA-b35742,OceanDNA-b35742_00388_705_3_27,403,SSPTKVQRTGWGYPTGKIFFILTDLNFLENGMDQSARYADLSLDED...,/scratch/users/ajaffe/deepeco/data/odna_redund...,OceanDNA-b35742_00388
20,OceanDNA-b16381,OceanDNA-b16381_00013_5941_1_86,305,QELSMAKKYDAGVKEYRDTYFTPDYVPLDTDLLACFKCTGQEGVPK...,/scratch/users/ajaffe/deepeco/data/odna_redund...,OceanDNA-b16381_00013


In [127]:
for key, row in rub_df_refiltered.iterrows():
    
    with open(rootdir + "genomes/second_pass/" + row["genome"] + ".fa", "w") as out:
        
        if ".gz" in row["gpath"]:
            with gzip.open(row["gpath"], "rt") as handle:
                for record in sfp(handle):
                    out.write(">%s\n%s\n" %(record[0], str(record[1])))
        else:
            for record in sfp(open(row["gpath"])):
                out.write(">%s\n%s\n" %(record[0], str(record[1])))  

In [129]:
existing_quality = pd.read_csv(rootdir + "genomes/first_pass_drep/genomeInformation.csv")
new_quality = existing_quality[existing_quality["genome"].isin([os.path.basename(item) \
                for item in glob.glob(rootdir + "genomes/second_pass/*")])]
new_quality.to_csv(rootdir + "genomes/second_pass_drep/genomeInformation.csv", index=False)

In [131]:
call = "dRep dereplicate %s -sa 0.95 -p 20 -comp 50 -con 10 -d -g %s --genomeInfo %s" %(rootdir + \
    "/genomes/second_pass_drep/", rootdir + "/genomes/second_pass/*fa",rootdir + "genomes/second_pass_drep/genomeInformation.csv")
cmd = sbatch("drep", 20, call)
print(cmd)

sbatch -J drep -p serc -t 1- -c 20 --mem 160G --wrap 'dRep dereplicate /scratch/users/ajaffe/deepeco//genomes/second_pass_drep/ -sa 0.95 -p 20 -comp 50 -con 10 -d -g /scratch/users/ajaffe/deepeco//genomes/second_pass/*fa --genomeInfo /scratch/users/ajaffe/deepeco/genomes/second_pass_drep/genomeInformation.csv'


## compile and export

In [7]:
cmdir(rootdir + "/tables")

In [20]:
cdb = pd.read_csv(rootdir + "genomes/second_pass_drep/data_tables/Cdb.csv")
cdb["user_genome"] = cdb["genome"].apply(lambda x: x.replace(".fa", ""))
cdb["representative"] = cdb["genome"].apply(lambda x: x in \
    [os.path.basename(item) for item in glob.glob(rootdir + "genomes/second_pass_drep/dereplicated_genomes/*")])
cdb = cdb[["user_genome", "primary_cluster", "secondary_cluster", "representative"]]
cdb.head(5)

,user_genome,primary_cluster,secondary_cluster,representative
0,OceanDNA-b21631,1,1_0,True
1,TARA_SAMEA2619970_METAG_OGBJMBMB,2,2_0,True
2,OceanDNA-b21628,3,3_0,True
3,OceanDNA-b26756,4,4_1,False
4,TARA_SAMEA4397330_METAG_GFBBFFPE,4,4_1,True


In [21]:
base = gtresults[["user_genome", "classification"]]
base = base.merge(checkm_df, how="left")
base = base.merge(cdb, how="right").sort_values(["primary_cluster", "secondary_cluster", \
            "representative"], ascending=[True,True,False]).rename(columns={"user_genome":"genome"}).fillna("not clustered")
base.to_csv(rootdir + "tables/redundant_genome_info.tsv", sep="\t", index=False)
base.head(5)

,genome,classification,checkm_completeness,checkm_redundancy,primary_cluster,secondary_cluster,representative
0,OceanDNA-b21631,d__Bacteria;p__Patescibacteria;c__Gracilibacte...,82.99,2.48,1,1_0,True
1,TARA_SAMEA2619970_METAG_OGBJMBMB,d__Bacteria;p__Patescibacteria;c__Gracilibacte...,50.57,0.99,2,2_0,True
2,OceanDNA-b21628,d__Bacteria;p__Patescibacteria;c__Gracilibacte...,79.21,2.09,3,3_0,True
4,TARA_SAMEA4397330_METAG_GFBBFFPE,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,88.16,1.10,4,4_1,True
3,OceanDNA-b26756,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,73.75,1.47,4,4_1,False


In [24]:
cmd = "rclone copy /scratch/users/ajaffe/deepeco/tables/redundant_genome_info.tsv drive:deepeco/tables --progress"
print(cmd)

rclone copy /scratch/users/ajaffe/deepeco/tables/redundant_genome_info.tsv drive:deepeco/tables --progress
